In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode

from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
from syft.core.frameworks.torch.tensor import _MPCTensor
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

In [29]:
x = torch.LongTensor([[1,-2],[-3,-4]])
x = x.share(bob, alice)

y = torch.LongTensor([[5,6],[7,8]])
y = y.share(bob, alice)

In [30]:
(x*y).get()


  5 -12
-21 -32
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [31]:
(-x).get()


-1  2
 3  4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [32]:
(y - x).get()


  4   8
 10  12
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [33]:
(x - y).get()


 -4  -8
-10 -12
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [12]:
x = torch.LongTensor([[1, 2], [3, 4]])
y = torch.LongTensor([[5, 6], [7, 8]])

x = x.share(bob, alice)
y = y.share(bob, alice)

z = x.mm(y)

value = z.child.shares.child.sum_get() % spdz.field

if (value > spdz.torch_max_value).all(): # TODO: value per value
    out = value - spdz.torch_field
else:
    out = value
out


 1.5629e+09  1.9658e+09
 9.5714e+07  1.3171e+09
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [ ]:
value = self.shares.child.sum_get() % spdz.field

gate = (value > spdz.torch_max_value).long()

neg_nums = (value - spdz.torch_field) * gate
pos_nums = value * (1 - gate)
result = neg_nums + pos_nums

In [10]:


assert (x.mm(y).get() - torch.LongTensor([[18, 22], [43, 49]])).abs().sum() < 5

x = torch.LongTensor([[1, -2], [3, -4]])
y = torch.LongTensor([[5, 6], [7, 8]])

target = x.mm(y)

x = x.share(bob, alice)
y = y.share(bob, alice)

result = x.mm(y)
assert (result.get() - target).abs().sum() < 5

AssertionError: 

In [34]:
x = torch.LongTensor([[1,-2],[3,-4]])
y = torch.LongTensor([[5,6],[7,8]])

target = x.mm(y)

x = x.share(bob, alice)
y = y.share(bob, alice)

result = x.mm(y)
(result.get() - target).abs().sum()<5

In [61]:
x = torch.LongTensor([[-1, -2], [-3, -4]])

x = x.share(bob, alice)

z = -x
# assert (z.get() == torch.LongTensor([[-1, -2], [3, 4]])).all()

z.get()


-1 -2
-3 -4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]


 1  2
 3  4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [36]:
target


 -9 -10
-13 -14
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

True

In [16]:
z.get()


 4  4
 4  4
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]